# geojson file

## Aim: Get a geojson file for future use in Choropleth

The structure of this notebooks is as follows:
- Download the file from http://gis.cityofathens.gr
- Filter out irrelevant areas
- Convert downloaded file's projection from ESPG:2100 to ESPG:4326
- Export the final geojson file

For each step we save our datasets into temporary geojson files.

In [1]:
import json
import pyproj
import os.path
import requests
import geopandas as gpd

**Downloading**

In [2]:
geojson_directory = os.path.join('..', 'Data', 'geojson')
url = 'http://gis.cityofathens.gr/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aper_enot_attki&outputFormat=json&srs=EPSG%3A2100&srsName=EPSG%3A2100'

response = requests.get(url)
if response.status_code == 200:
    with open(os.path.join(geojson_directory, 'dimoi_athens_full.geojson'), 'wb') as file:
        file.write(response.content)
    print('GeoJSON file downloaded successfully.')
else:
    print('Failed to download the GeoJSON file.')

GeoJSON file downloaded successfully.


**Cleaning**

In [3]:
geojson_full = gpd.read_file(os.path.join(geojson_directory, 'dimoi_athens_full.geojson'))
geojson_clean = geojson_full[~geojson_full['enotites'].isin(['ΠΕΙΡΑΙΩΣ','ΝΗΣΩΝ','ΑΝΑΤΟΛΙΚΗΣ ΑΤΤΙΚΗΣ','ΔΥΤΙΚΗΣ ΑΤΤΙΚΗΣ'])].reset_index()[["kwd_ypes","geometry"]]
geojson_clean.to_file(os.path.join(geojson_directory, 'geojson_clean.geojson'), driver='GeoJSON')

**Converting projection system from ESPG:2100 to ESPG:4326**

In [4]:
with open(os.path.join(geojson_directory, 'geojson_clean.geojson')) as f:
    data = json.load(f)

In [5]:
# Define the source and target coordinate systems
src_crs = pyproj.CRS.from_epsg(2100)
target_crs = pyproj.CRS.from_epsg(4326)

# Create a transformer to convert coordinates
transformer = pyproj.Transformer.from_crs(src_crs, target_crs, always_xy=True)

# Convert the coordinates in the GeoJSON features
for feature in data['features']:
    geometry = feature['geometry']
    coordinates = geometry['coordinates']

    def convert_coordinates(coords):
        if isinstance(coords[0], list):
            # If it's a MultiPolygon, recursively convert each set of coordinates
            return [convert_coordinates(sub_coords) for sub_coords in coords]
        else:
            # Convert the coordinates
            x, y = coords
            lon, lat = transformer.transform(x, y)
            return [lon, lat]

    converted_coords = convert_coordinates(coordinates)
    geometry['coordinates'] = converted_coords

# Save the converted GeoJSON to a new file
with open(os.path.join(geojson_directory, 'dimoi_final.geojson'), 'w') as f:
    json.dump(data, f)